===============================================================================================================

In [4]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import re

BASE = "https://manshurat.org"
START_PAGE = "https://manshurat.org/taxonomy/term/16?sort=search_api_aggregation_1%20DESC&search_api_views_fulltext=&field_document_official_year=&field_document_official_number=&field_field_sector%5B0%5D=66"
OUTPUT_DIR = "manshurat_pdfs"
HEADERS = {"User-Agent": "pdf-downloader/1.0"}

os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_article_links(page_url):
    """Collect article links inside the category page"""
    resp = requests.get(page_url, headers=HEADERS, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    links = []
    for a in soup.select("h2 a, div.views-field-title a, article a"):
        href = a.get("href")
        if href:
            full = urljoin(BASE, href)
            links.append(full)
    return links

def get_pdf_links(article_url):
    """Collect PDF links inside the article page"""
    resp = requests.get(article_url, headers=HEADERS, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    links = []

    # Check for Google Docs iframe viewer
    for iframe in soup.find_all("iframe", src=True):
        src = iframe["src"]
        if "docs.google.com/gview" in src:
            match = re.search(r"url=(.+)", src)
            if match:
                pdf_url = match.group(1)
                pdf_url = requests.utils.unquote(pdf_url)
                links.append(pdf_url)

    # Check for direct PDF links
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.lower().endswith(".pdf"):
            links.append(urljoin(article_url, href))
    return links

def download_pdf(pdf_url):
    """Download PDF file"""
    try:
        filename = os.path.basename(pdf_url)
        filename = re.sub(r'[^\w\-.()]', '_', filename)
        path = os.path.join(OUTPUT_DIR, filename)

        if os.path.exists(path):
            print(f"Skipped: {filename}")
            return

        resp = requests.get(pdf_url, headers=HEADERS, stream=True, timeout=60)
        resp.raise_for_status()

        with open(path, "wb") as f:
            for chunk in resp.iter_content(chunk_size=8192):
                f.write(chunk)

        size = os.path.getsize(path)
        print(f"Saved: {filename} ({size:,} bytes)")
    except Exception as e:
        print(f"Failed download: {pdf_url} - {e}")

# ====== Collecting Section Pages ======
visited_pages = set()
to_visit_pages = [START_PAGE]

while to_visit_pages:
    page_url = to_visit_pages.pop(0)

    if page_url in visited_pages:
        continue

    visited_pages.add(page_url)
    print(f"🔍 Visiting section page: {page_url}")

    try:
        articles = get_article_links(page_url)

        for article in articles:
            print(f"📄 Processing article: {article}")

            try:
                pdfs = get_pdf_links(article)
                for pdf in pdfs:
                    download_pdf(pdf)  # Download directly while collecting links
            except Exception as e:
                print(f"❌ Failed to process article: {article} - {e}")

        # Check for pagination
        resp = requests.get(page_url, headers=HEADERS, timeout=30)
        soup = BeautifulSoup(resp.text, "html.parser")

        for a in soup.select("ul.pagination li a[href]"):
            next_page = urljoin(BASE, a["href"])
            if next_page not in visited_pages and next_page not in to_visit_pages:
                to_visit_pages.append(next_page)

    except Exception as e:
        print(f"❌ Failed to process section page: {page_url} - {e}")

    time.sleep(1)

print("✅ All PDFs have been successfully downloaded")


🔍 زيارة صفحة القسم: https://manshurat.org/taxonomy/term/16?sort=search_api_aggregation_1%20DESC&search_api_views_fulltext=&field_document_official_year=&field_document_official_number=&field_field_sector%5B0%5D=66
📄 معالجة المقال: https://manshurat.org/content/nqt-syr-lkhswm-fy-dw-ttlb-lhkm-bdm-dstwry-3-mwd-blqnwn-lmdny
Saved: 5-brqm_87_lsn_41_qdyy.pdf (159,118 bytes)
📄 معالجة المقال: https://manshurat.org/content/dm-dstwry-fqr-bqnwn-lmrft-lmdny-wltjry-mnht-qd-lmhkm-musdr-lhkm-hq-nzr-lltms
Saved: 10-brqm_95_lsn_43_qdyy.pdf (159,951 bytes)
📄 معالجة المقال: https://manshurat.org/content/drj-md-syd-syd-wd-llh-lmhkwm-lyh-gybyan-fy-ljny-rqm-5858-lsn-2014-l-qym-lrhbyyn-lmd-khms
Saved: ldd_2_tb_-_tmyn-3.pdf (99,721 bytes)
📄 معالجة المقال: https://manshurat.org/content/drj-20-mthman-wshrktyn-wjm-lkhwn-lmslmsn-l-qwym-lrhbyyn-wlkynt-lrhby-lmd-khms-snwt
Saved: ldd_2_tb_-_tmyn-4-5.pdf (123,305 bytes)
📄 معالجة المقال: https://manshurat.org/content/rfd-tn-fy-dstwry-ns-bqnwn-lmrft-lmdny-wltjry-ytlq-b

In [5]:
import shutil

shutil.make_archive("downloads_zip", 'zip', "manshurat_pdfs")

'/content/downloads_zip.zip'

In [6]:
from google.colab import files

files.download("downloads_zip.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>